#  Import Libraries and loading dataset

In [11]:
import pandas as pd 

pd.set_option('display.max_rows', None)

df = pd.read_csv("files/salary_data.csv", ).drop('Unnamed: 0', axis=1)


## Data Analysis and cleaning

In [12]:
df.head(2)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1


#### renameing dataframe columns to be lower canse and words separated with '_'

In [13]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_title          742 non-null    object 
 1   salary_estimate    742 non-null    object 
 2   job_description    742 non-null    object 
 3   rating             742 non-null    float64
 4   company_name       742 non-null    object 
 5   location           742 non-null    object 
 6   headquarters       742 non-null    object 
 7   size               742 non-null    object 
 8   founded            742 non-null    int64  
 9   type_of_ownership  742 non-null    object 
 10  industry           742 non-null    object 
 11  sector             742 non-null    object 
 12  revenue            742 non-null    object 
 13  competitors        742 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 81.3+ KB


In [15]:
# dropping the rows there salary is missing 

df = df[df['salary_estimate'] != '-1']
df.shape

(742, 14)

In [16]:
# In salary_estimate column, there are some records per hour.

df['hourly'] = df['salary_estimate'].apply(lambda x:1 if 'per hour' in x.lower() else 0)
df.hourly.value_counts()

0    718
1     24
Name: hourly, dtype: int64

In [17]:
df['employer_provided'] = df['salary_estimate'].apply(lambda x:1 if 'employer provided salary' in x.lower() else 0)
df.employer_provided.value_counts()

0    725
1     17
Name: employer_provided, dtype: int64

In [18]:
# cleaning salary_estimate so we can calcuate the avg salary (which will be our label)

df['salary_estimate'] = df['salary_estimate'].apply(lambda x: x.replace('K', '').replace('$',''))
df['salary_estimate'] = df['salary_estimate'].apply(lambda x: x.split('(')[0].strip())
df['salary_estimate'] = df['salary_estimate'].apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))


In [19]:
df['min_salary'] = df['salary_estimate'].apply(lambda x: x.split('-')[0])
df['max_salary'] = df['salary_estimate'].apply(lambda x: x.split('-')[1])
df['min_salary'] = df.min_salary.astype('int')
df['max_salary'] = df.max_salary.astype('int')

#hourly wage to annual 

df['min_salary'] = df.apply(lambda x: x.min_salary*2 if x.hourly ==1 else x.min_salary, axis =1)
df['max_salary'] = df.apply(lambda x: x.max_salary*2 if x.hourly ==1 else x.max_salary, axis =1)

In [20]:
# df['salary_estimate']
# df['min_salary']

In [21]:
df['avg_salary'] = (df['min_salary'] + df['max_salary']) / 2

In [22]:
df.head()

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,hourly,employer_provided,min_salary,max_salary,avg_salary
0,Data Scientist,53-91,"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,72.0
1,Healthcare Data Scientist,63-112,What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,87.5
2,Data Scientist,80-90,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,85.0
3,Data Scientist,56-97,*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,76.5
4,Data Scientist,86-143,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5


In [12]:
df['company_name'] = df.apply(lambda x: x['company_name'] if x.rating < 0 else x['company_name'][:-4], axis=1)

In [13]:
df['job_state'] = df.location.apply(lambda x:x.split(',')[1])
df['same_state'] = df.apply(lambda x: 1 if x.location == x.headquarters else 0, axis = 1)
df['age'] = df.founded.apply(lambda x: x if x <1 else 2021 - x)

In [14]:
# Fix state Los Angeles 

df['job_state']= df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'los angeles' else 'CA')
df.job_state.value_counts()

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
WA     21
NC     21
NJ     17
FL     16
OH     14
TN     13
DC     11
CO     11
WI     10
UT     10
IN     10
MO      9
AZ      9
AL      8
GA      6
KY      6
MI      6
DE      6
IA      5
CT      5
NE      4
LA      4
OR      4
KS      3
NM      3
MN      2
ID      2
SC      1
RI      1
Name: job_state, dtype: int64

In [15]:
df['python'] = df.job_description.apply(lambda x: 1 if 'python' in x.lower() else 0)
df.python.value_counts()

1    392
0    350
Name: python, dtype: int64

In [16]:
# df['r'] = df.job_description.apply(lambda x: 1 if 'r studio' in x.lower() or ' r ' in x.lower() else 0)
# df.r.value_counts()

In [17]:
df['aws'] = df.job_description.apply(lambda x: 1 if 'aws' in x.lower() else 0)
df.aws.value_counts()

0    566
1    176
Name: aws, dtype: int64

In [18]:
df['spark'] = df.job_description.apply(lambda x: 1 if 'spark' in x.lower() or 'pyspark' in x.lower()  else 0)
df.spark.value_counts()

0    575
1    167
Name: spark, dtype: int64

In [19]:
df['excel'] = df.job_description.apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.excel.value_counts()

1    388
0    354
Name: excel, dtype: int64

In [24]:
# simplifie the title so we can have like 5 categories

def title_simplifier(title):
    title = title.lower()
    if 'scientist' in title:
        return 'data scientist'
    elif 'data engineer' in title:
        return 'data engineer'
    elif 'analyst' in title:
        return 'analyst'
    elif 'machine learning' in title:
        return 'mle'
    elif 'manager' in title:
        return 'manager'
    elif 'director' in title:
        return 'director'
    else:
        return 'na'
    

def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'jr'
    else:
        return 'na'

In [25]:
df['job_simp'] = df.job_title.apply(title_simplifier)
df['seniority'] = df.job_title.apply(seniority)

In [26]:
df['desc_len'] = df.job_description.apply(lambda x: len(x))

In [27]:
df['num_comp'] = df.competitors.apply(lambda x: len(x.split(',')) if x != '-1' else 0 )

In [28]:
df.columns

Index(['job_title', 'salary_estimate', 'job_description', 'rating',
       'company_name', 'location', 'headquarters', 'size', 'founded',
       'type_of_ownership', 'industry', 'sector', 'revenue', 'competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'job_simp', 'seniority', 'desc_len', 'num_comp'],
      dtype='object')

In [29]:
df.head()

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,...,competitors,hourly,employer_provided,min_salary,max_salary,avg_salary,job_simp,seniority,desc_len,num_comp
0,Data Scientist,53-91,"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,...,-1,0,0,53,91,72.0,data scientist,na,2536,0
1,Healthcare Data Scientist,63-112,What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,...,-1,0,0,63,112,87.5,data scientist,na,4783,0
2,Data Scientist,80-90,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,...,-1,0,0,80,90,85.0,data scientist,na,3461,0
3,Data Scientist,56-97,*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,...,"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,76.5,data scientist,na,3883,3
4,Data Scientist,86-143,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5,data scientist,na,2728,3


In [30]:
df.to_csv('files/salary_data_cleaned.csv', index=False)